**Requirements**

In [ ]:
!pip install segmentation-models
!pip install tensorflow==2.13.0
!pip install keras==2.13.1
#For live loss function updates
!pip install livelossplot
!pip install h5py==3.8.0

**Required Libraries**

In [ ]:
%matplotlib inline

In [ ]:
import glob
import cv2
import os
import numpy as np
from natsort import natsorted
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from livelossplot import PlotLossesKerasTF


**Mounting the Drive**

In [ ]:
# Mount Google Drive if needed
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Setting Up Environment for Segmentation Models**

In [ ]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"
import tensorflow as tf
import segmentation_models as sm
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

**Data Preprocessing and Loading**

In [ ]:
#print(os.listdir("membrane/train"))

#Resizing images is optional, CNNs are ok with large images
SIZE_X = 128 #Resize images (height  = X, width = Y)
SIZE_Y = 128
OGX = 3000
OGY = 1700

#Capture training image info as a list
train_images = []

for directory_path in glob.glob("/content/drive/MyDrive/Sclera_Project_Files/ss_train/images"):
    for img_path in sorted(glob.glob(os.path.join(directory_path, "*png"))) :
        #print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR )   #cv2.IMREAD_COLOR
        img = cv2.resize(img, (SIZE_Y, SIZE_X))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        train_images.append(img)
        # train_labels.append(label)
#Convert list to array for machine learning processing


train_images = np.array(train_images)

#Capture mask/label info as a list
train_masks = []
for directory_path in glob.glob("/content/drive/MyDrive/Sclera_Project_Files/ss_train/masks"):
    for mask_path in sorted(glob.glob(os.path.join(directory_path, "*.png"))):
        #print(mask_path)
        mask = cv2.imread(mask_path, 0)   #,0
        mask = cv2.resize(mask, (SIZE_Y, SIZE_X))
        # mask = cv2.cvtColor(mask, cv2.COLOR_RGB2BGR) #
        train_masks.append(mask)
        # train_labels.append(label)
#Convert list to array for machine learning processing
train_masks = np.array(train_masks)


**Splitting Data into Training and Validation Sets**

In [ ]:
X = train_images
Y = train_masks
Y = np.expand_dims(Y, axis=3) #May not be necessary.. leftover from previous code


from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

# preprocess input
x_train = preprocess_input(x_train)
x_val = preprocess_input(x_val)

**Data Type Conversion**

In [ ]:
#Added
y_train = y_train.astype("float32")
y_val = y_val.astype("float32")

**Model Initialization and Compilation**

In [ ]:
model = sm.Unet(BACKBONE, encoder_weights='imagenet')
model.compile('Adam', loss=sm.losses.bce_jaccard_loss, metrics=[sm.metrics.iou_score],)

print(model.summary())


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 data (InputLayer)           [(None, None, None, 3)]      0         []                            
                                                                                                  
 bn_data (BatchNormalizatio  (None, None, None, 3)        9         ['data[0][0]']                
 n)                                                                                               
                                                                                                  
 zero_padding2d_34 (ZeroPad  (None, None, None, 3)        0         ['bn_data[0][0]']             
 ding2D)                                                                                          
                                                                                            

**Model Training**

In [ ]:
# Train the model
from livelossplot import PlotLossesKeras
#Include this as callback., but slows the training (callbacks=[PlotLossesKeras()],)
model.fit(
   x=x_train,
   y=y_train,
   batch_size=30,
   epochs=60,
   verbose=1,
   validation_data=(x_val, y_val),
)
accuracy = model.evaluate(x_val, y_val)
print("Validation Accuracy:", accuracy)

Epoch 1/60
4/4 [==============================] - 37s 9s/step - loss: -1666.5902 - iou_score: 139.3965 - val_loss: -1218.9623 - val_iou_score: 2.1527
Epoch 2/60
4/4 [==============================] - 38s 9s/step - loss: -1719.8278 - iou_score: 137.7306 - val_loss: -1714.3055 - val_iou_score: 2.7421
Epoch 3/60
4/4 [==============================] - 35s 9s/step - loss: -1782.1777 - iou_score: 137.1013 - val_loss: -3488.9939 - val_iou_score: 40.3779
Epoch 4/60
4/4 [==============================] - 37s 9s/step - loss: -1852.9731 - iou_score: 152.5081 - val_loss: -1429.6855 - val_iou_score: 3.4457
Epoch 5/60
4/4 [==============================] - 35s 9s/step - loss: -1919.3560 - iou_score: 149.9965 - val_loss: -1471.5031 - val_iou_score: 9.6486
Epoch 6/60
4/4 [==============================] - 37s 9s/step - loss: -1991.0776 - iou_score: 158.4286 - val_loss: -2389.9055 - val_iou_score: 29.5918
Epoch 7/60
4/4 [==============================] - 35s 8s/step - loss: -2062.2034 - iou_score: 151.

In [ ]:
# Save the trained model
model.save('/content/drive/MyDrive/Sclera_Project_Files/Models/Sclera_180.h5')